In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote

driver = webdriver.Chrome('data/chromedriver.exe')

base_url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query='
url = base_url + quote('테슬라')
driver.get(url)

In [4]:
driver.find_element_by_css_selector("div.lnb_group > div > ul > li:nth-child(4) > a").click()
time.sleep(5)

In [5]:
# blog 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [6]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [7]:
lis = soup.select('li.bx._svp_item')
len(lis)

88

In [8]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.sub_time.sub_txt').get_text()
title, date, content

('테슬라 모델3 PPF 찾고 계시나요? [부산 제로맥스]',
 '2021.08.14.',
 '오늘 만나볼 차량은 테슬라 모델3 리프레시입니다. 요즘 정말 핫한 차량이죠? 오늘 당장 계약해도 언제 출고... 해당 테슬라 모델 3 리프레시 차량처럼 기존에 시공된 생활보호 패키지의 경우는 그 차이가 심하진 않겠지만...')

In [9]:
lis = soup.select('li.bx._svp_item')
len(lis)

88

In [10]:
title_list, content_list, date_list = [],[],[]
for i, li in enumerate(lis):
    try:
        adv = li.select_one('a.link_ad').get_text()
        print(adv.strip())
        if adv.strip() == '광고':            
            continue
    except:
        title = li.select_one('a.api_txt_lines.total_tit').get_text()
        content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
        date = li.select_one('.sub_time.sub_txt').get_text()

        title_list.append(title)
        content_list.append(content)
        date_list.append(date)

In [11]:
df = pd.DataFrame({
    'title':title_list, 'date':date_list, 'content':content_list
})
df.head()

,title,date,content
0,테슬라 모델3 PPF 찾고 계시나요? [부산 제로맥스],2021.08.14.,오늘 만나볼 차량은 테슬라 모델3 리프레시입니다. 요즘 정말 핫한 차량이죠? 오늘 ...
1,"폐배터리 관련주 테슬라 삼기, 천보, 영화테크 주가 강세 이어갈까?",2021.08.20.,첫번째는 테슬라 관련주인 삼기 ( 122350 ) 입니다. 알루미늄 다이캐스팅 전문...
2,"증시리뷰 (테슬라 ai데이, 자율주행관련주)",2021.08.16.,테슬라 ai 데이 8/19 - 자율주행 관련주 관심 2. 8/18 FOMC 회의록 ...
3,테슬라 주가가 헤매고 있는 이유,2021.08.13.,테슬라 주주들의 불면증이 날로 심해지고 있다고 하네요. 이유는 올초까지만 해도 화성...
4,럭셔리 하우스에 설치되는 테슬라의 에코 시스템,5일 전,"테슬라의 세 번째 주거 커뮤니티 프로젝트 환경 보호에 대한 관심이 높아지면서, 점차..."


In [12]:
df.shape

(88, 3)

In [13]:
df.to_csv('data/tesla.txt')

In [14]:
driver.close()